# **ML Assignment 7**
Submitted by Kshitij Mittal

**PART A: Recurrent Neural Network & Classification:**

The objective is to detect the security breach by predicting suspicious access using an RNN model and the provided Logfile data.

Logfile data includes login information like LogID, Timestamp, Method, Path, Status Code, Source, Remote Address, User Agent etc. The last indicator in each row denotes breach(1) and no breach(0) which is the target variable.



---
### Part 1. Data Processing

In [107]:
import sys
import os
import json
import pandas as pd
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding
# from keras.layers.Embedding import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

from tqdm import tqdm

In [52]:
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.sequence import pad_sequences

In [92]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [14]:
dataframe = pd.read_csv("/Users/kshitijmittal/Documents/UChicago Acad/uchicago_codebase/06_MachineLearning/06_RNNs_LSTMs/00_Data/dev-access.csv", engine='python', quotechar='|', header=None)
dataset = dataframe.values

In [15]:
# Shape of the data set
dataset.shape

(26773, 2)

In [16]:
pd.options.display.max_colwidth=1000
dataframe.head()

,0,1
0,"{""timestamp"":1502738402847,""method"":""post"",""query"":{},""path"":""/login"",""statusCode"":401,""source"":{""remoteAddress"":""88.141.113.237"",""referer"":""http://localhost:8002/enter""},""route"":""/login"",""headers"":{""host"":""localhost:8002"",""accept-language"":""en-us"",""accept-encoding"":""gzip, deflate"",""connection"":""keep-alive"",""accept"":""*/*"",""referer"":""http://localhost:8002/enter"",""cache-control"":""no-cache"",""x-requested-with"":""XMLHttpRequest"",""content-type"":""application/json"",""content-length"":""36""},""requestPayload"":{""username"":""Carl2"",""password"":""bo""},""responsePayload"":{""statusCode"":401,""error"":""Unauthorized"",""message"":""Invalid Login""}}",0
1,"{""timestamp"":1502738402849,""method"":""post"",""query"":{},""path"":""/login"",""statusCode"":401,""source"":{""remoteAddress"":""88.141.113.237""},""route"":""/login"",""headers"":{""host"":""localhost:8002"",""connection"":""keep-alive"",""cache-control"":""no-cache"",""accept"":""*/*"",""accept-encoding"":""gzip, deflate, br"",""accept-language"":""en-US,en;q=0.8,es;q=0.6"",""content-type"":""application/json"",""content-length"":""47""},""requestPayload"":{""username"":""pafzah"",""password"":""worldburn432""},""responsePayload"":{""statusCode"":401,""error"":""Unauthorized"",""message"":""Invalid Login""}}",0
2,"{""timestamp"":1502738402852,""method"":""post"",""query"":{},""path"":""/login"",""statusCode"":401,""source"":{""remoteAddress"":""205.49.83.118""},""route"":""/login"",""headers"":{""host"":""localhost:8002"",""connection"":""keep-alive"",""cache-control"":""no-cache"",""accept"":""*/*"",""accept-encoding"":""gzip, deflate, br"",""accept-language"":""en-US,en;q=0.8,es;q=0.6"",""content-type"":""application/json"",""content-length"":""44""},""requestPayload"":{""username"":""Panos1"",""password"":""najrijkom""},""responsePayload"":{""statusCode"":401,""error"":""Unauthorized"",""message"":""Invalid Login""}}",0
3,"{""timestamp"":1502738402852,""method"":""post"",""query"":{},""path"":""/login"",""statusCode"":401,""source"":{""remoteAddress"":""205.49.83.118"",""referer"":""http://localhost:8002/enter""},""route"":""/login"",""headers"":{""host"":""localhost:8002"",""accept-language"":""en-us"",""accept-encoding"":""gzip, deflate"",""connection"":""keep-alive"",""accept"":""*/*"",""referer"":""http://localhost:8002/enter"",""cache-control"":""no-cache"",""x-requested-with"":""XMLHttpRequest"",""content-type"":""application/json"",""content-length"":""47""},""requestPayload"":{""username"":""vuvpuvehu"",""password"":""password1""},""responsePayload"":{""statusCode"":401,""error"":""Unauthorized"",""message"":""Invalid Login""}}",0
4,"{""timestamp"":1502738402853,""method"":""post"",""query"":{},""path"":""/login"",""statusCode"":401,""source"":{""remoteAddress"":""137.196.95.116""},""route"":""/login"",""headers"":{""host"":""localhost:8002"",""connection"":""keep-alive"",""cache-control"":""no-cache"",""accept"":""*/*"",""accept-encoding"":""gzip, deflate, br"",""accept-language"":""en-US,en;q=0.8,es;q=0.6"",""content-type"":""application/json"",""content-length"":""41""},""requestPayload"":{""username"":""Michele"",""password"":""mokgu""},""responsePayload"":{""statusCode"":401,""error"":""Unauthorized"",""message"":""Invalid Login""}}",0


In [21]:
X = dataset[:,0]
Y = dataset[:,1]
Y

array([0, 0, 0, ..., 1, 1, 1], dtype=object)

In [22]:
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))

In [25]:
X

array(['{"method":"post","query":{},"path":"/login","statusCode":401,"requestPayload":{"username":"Carl2","password":"bo"}}',
       '{"method":"post","query":{},"path":"/login","statusCode":401,"requestPayload":{"username":"pafzah","password":"worldburn432"}}',
       '{"method":"post","query":{},"path":"/login","statusCode":401,"requestPayload":{"username":"Panos1","password":"najrijkom"}}',
       ...,
       '{"method":"post","query":{},"path":"/checkout","statusCode":400,"requestPayload":{"creditCard":"<script src=\\"http://attacker/malicious\\u2011script.js\\"></script>"}}',
       '{"method":"post","query":{},"path":"/checkout","statusCode":400,"requestPayload":{"creditCard":"<meta http-equiv=\\"refresh\\">"}}',
       '{"method":"post","query":{},"path":"/checkout","statusCode":400,"requestPayload":{"creditCard":"<meta http-equiv=\\"refresh\\">"}}'],
      dtype=object)

In [26]:
# We next will tokenize our data, which just means vectorizing our text. 
# Given the data we will tokenize every character (thus char_level = True)

tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)


In [110]:
tokenizer.word_index

{'"': 1,
 'e': 2,
 't': 3,
 ':': 4,
 'a': 5,
 's': 6,
 'o': 7,
 'u': 8,
 'r': 9,
 ',': 10,
 'd': 11,
 'l': 12,
 'p': 13,
 'h': 14,
 'y': 15,
 'q': 16,
 'c': 17,
 '{': 18,
 '}': 19,
 'm': 20,
 'n': 21,
 '0': 22,
 '4': 23,
 'i': 24,
 '/': 25,
 'g': 26,
 ' ': 27,
 '2': 28,
 '1': 29,
 '&': 30,
 ';': 31,
 'w': 32,
 '3': 33,
 '=': 34,
 "'": 35,
 'k': 36,
 '\\': 37,
 '<': 38,
 '>': 39,
 'b': 40,
 'f': 41,
 'v': 42,
 '-': 43,
 '6': 44,
 '7': 45,
 '8': 46,
 '5': 47,
 'j': 48,
 'z': 49,
 '_': 50,
 '!': 51,
 '(': 52,
 '.': 53,
 '9': 54,
 'x': 55,
 '#': 56,
 ')': 57,
 '*': 58,
 '?': 59,
 '[': 60,
 ']': 61,
 '+': 62}

In [43]:
# Padding our data
max_log_length = 1024
X_processed = pad_sequences(X, maxlen=max_log_length)

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_processed,Y,train_size=0.75, test_size=0.25,random_state=1234)

In [111]:
X_train

array([[ 0,  0,  0, ..., 12, 12, 19],
       [ 0,  0,  0, ..., 12, 12, 19],
       [ 0,  0,  0, ..., 12, 12, 19],
       ...,
       [ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ...,  1, 19, 19],
       [ 0,  0,  0, ..., 12, 12, 19]], dtype=int32)

----
### Part 2. Model 1 - Baseline RNN

In [66]:
from keras import layers
base_rnn=keras.Sequential(
    [layers.Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length), ## Only the first layer needs an input dimension
     # We have 63 total words (num_words). We are converting them into dense vectors of size 32. Input length is constant, padded at 1024
     layers.SimpleRNN(units=32, activation='relu'),
     layers.Dense(units=1, activation='sigmoid')
    ]
)

base_rnn.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [67]:
base_rnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1024, 32)          2016      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


In [87]:
X_train_tens=tf.convert_to_tensor(X_train, dtype=tf.int32)
X_test_tens=tf.convert_to_tensor(X_test, dtype=tf.int32)
X_train_tens.shape


TensorShape([20079, 1024])

In [89]:
y_train
y_train_tens=tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test_tens=tf.convert_to_tensor(y_test, dtype=tf.int32)


In [91]:
# Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

history = base_rnn.fit(x=X_train_tens, y=y_train_tens, validation_split=0.25, epochs=3, batch_size=128)

Epoch 1/3
118/118 [==============================] - 23s 184ms/step - loss: 0.5393 - accuracy: 0.6864 - val_loss: 0.1870 - val_accuracy: 0.9311
Epoch 2/3
118/118 [==============================] - 22s 187ms/step - loss: 0.1522 - accuracy: 0.9444 - val_loss: 0.1285 - val_accuracy: 0.9544
Epoch 3/3
118/118 [==============================] - 23s 193ms/step - loss: 0.0844 - accuracy: 0.9693 - val_loss: 0.0534 - val_accuracy: 0.9823


In [113]:
history = base_rnn.fit(x=X_train_tens, y=y_train_tens, validation_split=0.25, epochs=3, batch_size=128)

Epoch 1/3
118/118 [==============================] - 21s 180ms/step - loss: 0.0694 - accuracy: 0.9774 - val_loss: 0.0393 - val_accuracy: 0.9892
Epoch 2/3
118/118 [==============================] - 21s 180ms/step - loss: 0.0332 - accuracy: 0.9905 - val_loss: 0.0268 - val_accuracy: 0.9926
Epoch 3/3
118/118 [==============================] - 21s 179ms/step - loss: 0.0294 - accuracy: 0.9927 - val_loss: 0.0794 - val_accuracy: 0.9713


In [93]:
# Number of data points through each epoch
118*128

# Number of training - validation data points
20079*0.75

15104

In [96]:
# Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

base_rnn.evaluate(
    x = X_test_tens, y = y_test_tens, batch_size=128, use_multiprocessing = True
    )

53/53 [==============================] - 2s 39ms/step - loss: 0.0526 - accuracy: 0.9804


[0.052604302763938904, 0.9804302453994751]

----
### Part 3: Model 2 - LSTM + Dropout Layers

In [103]:
model2=keras.Sequential([
    layers.Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length),
    layers.LSTM(units=64, recurrent_dropout=0.5),
    layers.Dropout(rate=0.5),
    layers.Dense(units=1, activation='sigmoid')
])

model2.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model2.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 1024, 32)          2016      
                                                                 
 lstm_4 (LSTM)               (None, 64)                24832     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________


In [105]:
# Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.
model2.fit(x=X_train_tens, y=y_train_tens, validation_split=0.25, epochs=3, batch_size=128, use_multiprocessing=True)

Epoch 1/3
118/118 [==============================] - 108s 914ms/step - loss: 0.6438 - accuracy: 0.6287 - val_loss: 0.5966 - val_accuracy: 0.6386
Epoch 2/3
118/118 [==============================] - 102s 867ms/step - loss: 0.3784 - accuracy: 0.8408 - val_loss: 0.1303 - val_accuracy: 0.9699
Epoch 3/3
118/118 [==============================] - 104s 883ms/step - loss: 0.1172 - accuracy: 0.9698 - val_loss: 0.1087 - val_accuracy: 0.9741


In [108]:
model2.evaluate(x = X_test_tens, y = y_test_tens, batch_size=128, use_multiprocessing = True)

53/53 [==============================] - 8s 142ms/step - loss: 0.0702 - accuracy: 0.9824


[0.07024699449539185, 0.9823722839355469]